In [1]:
import cv2
import mediapipe as mp
import numpy as np
import socket
from collections import deque

FOCAL_LENGTH_PX = 650
KNOWN_FACE_WIDTH_M = 0.14

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    model_complexity=1
)

cap = cv2.VideoCapture(0)

# Create UDP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address1 = ('127.0.0.1', 5051)
server_address2 = ('127.0.0.1', 5051)

window_size = 5
num_joints = 33  # MediaPipe Pose has 33 landmarks
joint_history = [deque(maxlen=window_size) for _ in range(num_joints)]


def smooth_joints(joint_list):
    smoothed = []
    for i, (x, y, z) in enumerate(joint_list):
        joint_history[i].append((x, y, z))
        avg_x = sum(p[0] for p in joint_history[i]) / len(joint_history[i])
        avg_y = sum(p[1] for p in joint_history[i]) / len(joint_history[i])
        avg_z = sum(p[2] for p in joint_history[i]) / len(joint_history[i])
        smoothed.append((round(avg_x, 5), round(avg_y, 5), round(avg_z, 7)))
    return smoothed


def main():
    print("Start detecting and sending data...")
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Failed to read a frame from the camera.")
            continue

        image = cv2.flip(image, 1)
        h, w, _ = image.shape
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = holistic.process(image_rgb)
        image.flags.writeable = True
        image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        all_body_landmarks_world = None

        if results.face_landmarks and results.pose_world_landmarks:
            face_landmarks = results.face_landmarks.landmark
            p_left, p_right = face_landmarks[234], face_landmarks[454]
            current_pixel_width = np.sqrt(
                (p_right.x * w - p_left.x * w) ** 2 + (p_right.y * h - p_left.y * h) ** 2
            )

            if current_pixel_width > 0:
                z_distance_m = (KNOWN_FACE_WIDTH_M * FOCAL_LENGTH_PX) / current_pixel_width
                anchor_face = face_landmarks[1]
                x_ratio, y_ratio = anchor_face.x - 0.5, anchor_face.y - 0.5
                x_world = x_ratio * (z_distance_m / FOCAL_LENGTH_PX) * w
                y_world = -y_ratio * (z_distance_m / FOCAL_LENGTH_PX) * h
                anchor_world_pos = np.array([x_world, y_world, z_distance_m])

                anchor_pose_relative = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.NOSE]
                anchor_pose_relative_pos = np.array(
                    [anchor_pose_relative.x, anchor_pose_relative.y, anchor_pose_relative.z]
                )
                skeleton_origin_world_pos = anchor_world_pos - anchor_pose_relative_pos

                all_body_landmarks_world = []
                for landmark in results.pose_world_landmarks.landmark:
                    landmark_relative_pos = np.array([landmark.x, landmark.y, landmark.z])
                    landmark_world_pos = skeleton_origin_world_pos + landmark_relative_pos
                    all_body_landmarks_world.append(landmark_world_pos)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
            )

        if all_body_landmarks_world is not None and results.pose_landmarks:
            for i, landmark_world in enumerate(all_body_landmarks_world):
                landmark_image = results.pose_landmarks.landmark[i]

                if landmark_image.visibility < 0.5:
                    continue

                pixel_x = int(landmark_image.x * w)
                pixel_y = int(landmark_image.y * h)

                x, y, z = landmark_world[0], landmark_world[1], landmark_world[2]
                text = f"X:{x:.2f} Y:{y:.2f} Z:{z:.2f}"

                cv2.putText(
                    image,
                    text,
                    (pixel_x + 10, pixel_y + 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.35,
                    (255, 255, 255),
                    1,
                    cv2.LINE_AA
                )

            # Convert 3D coordinates to a string and send
            smoothed_landmarks = smooth_joints(all_body_landmarks_world)

            coordinate_str = []
            for landmark in smoothed_landmarks:
                x, y, z = landmark
                coordinate_str.append(f"{x:.6f}")
                coordinate_str.append(f"{y:.6f}")
                coordinate_str.append(f"{z:.6f}")

            # Format: "x0,y0,z0,x1,y1,z1,..."
            data = ",".join(coordinate_str)

            # Send UDP data
            try:
                bytes_sent2 = sock.sendto(data.encode("utf-8"), server_address2)
                print(f"Sent to port {server_address2[1]} - bytes sent: {bytes_sent2}")
            except Exception as e:
                print(f"Failed to send UDP data: {e}")

        cv2.imshow("Real-time 3D Pose Annotation", image)

        if cv2.waitKey(5) & 0xFF == 27:
            break

    cap.release()
    sock.close()
    cv2.destroyAllWindows()
    holistic.close()


if __name__ == "__main__":
    main()


Start detecting and sending data...
Sent to port 5051 - bytes sent: 912
Sent to port 5051 - bytes sent: 911
Sent to port 5051 - bytes sent: 912
Sent to port 5051 - bytes sent: 915
Sent to port 5051 - bytes sent: 920
Sent to port 5051 - bytes sent: 922
Sent to port 5051 - bytes sent: 922
Sent to port 5051 - bytes sent: 921
Sent to port 5051 - bytes sent: 919
Sent to port 5051 - bytes sent: 919
Sent to port 5051 - bytes sent: 919
Sent to port 5051 - bytes sent: 919
Sent to port 5051 - bytes sent: 919
Sent to port 5051 - bytes sent: 919
Sent to port 5051 - bytes sent: 919
Sent to port 5051 - bytes sent: 919
Sent to port 5051 - bytes sent: 921
Sent to port 5051 - bytes sent: 922
Sent to port 5051 - bytes sent: 922
Sent to port 5051 - bytes sent: 922
Sent to port 5051 - bytes sent: 923
Sent to port 5051 - bytes sent: 921
Sent to port 5051 - bytes sent: 921
Sent to port 5051 - bytes sent: 920
Sent to port 5051 - bytes sent: 920
Sent to port 5051 - bytes sent: 920
Sent to port 5051 - bytes se